In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls


drive  sample_data


In [0]:
#%cd Metric\ Learning\ for\ Facial\ Descriptors/
#!ls

In [0]:
#!unzip metric-learning.zip

In [0]:
path= "/content/drive/My Drive/Hackathon RAAI/Metric Learning for Facial Descriptors/"

In [0]:
import numpy as np
import pandas as pd

In [0]:
train_set = np.load(path+"train_set.npy")
#train_labels.csv
train_labels = pd.read_csv(path+"train_labels.csv")
test_set = np.load(path+"test_set.npy")
test_pairs = pd.read_csv(path+"test_pairs.csv")

In [0]:
'''
for i in range(80):
  print(train_set[0][6*i:6*i+5])
  print(train_set[1][6*i:6*i+5])
  print("\n\n")
'''

'\nfor i in range(80):\n  print(train_set[0][6*i:6*i+5])\n  print(train_set[1][6*i:6*i+5])\n  print("\n\n")\n'

In [0]:
classes = {}
j=-1
for i in range (train_labels.shape[0]):
  if (train_labels["class"][i]>j):
    j = train_labels["class"][i]
    s = []
    classes[j]=s

  classes[j].append(i)  

In [9]:
print(classes[10])

[1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437]


In [0]:
min_class = 1000000
max_class = 0
for cl in classes:
  if max_class< len(classes[cl]):
    max_class = len(classes[cl])
  if min_class> len(classes[cl]):
    min_class = len(classes[cl])

In [0]:
from random import randrange as rg
import random

In [0]:
random.seed()

In [0]:
def get_same_person(classes):
  idx_max = len(classes)
  id1 = rg(idx_max)
  idx1 = rg(len(classes[id1]))
  idx2 = rg(len(classes[id1]))
  x1 = classes[id1][idx1]
  x2 = classes[id1][idx2]
  return [x1,x2]
  

In [0]:
def get_different_person(classes):
  idx_max = len(classes)
  id1 = rg(idx_max)
  id2 = rg(idx_max)
  if(id1 == id2):
    id1 = (id1+10)%idx_max
  idx1 = rg(len(classes[id1]))
  idx2 = rg(len(classes[id2]))
  x1 = classes[id1][idx1]
  x2 = classes[id2][idx2]
  return [x1,x2]

In [0]:
def generate_samples(num_of_samples, classes=classes):
  
  samples = []
  for i in range(int(num_of_samples/10)):
    samples.append([get_same_person(classes), 0])
    samples.append([get_same_person(classes), 0])
    samples.append([get_same_person(classes), 0])
    samples.append([get_same_person(classes), 0])
    samples.append([get_same_person(classes), 0])
    
    samples.append([get_different_person(classes),1])
    samples.append([get_different_person(classes),1])
    samples.append([get_different_person(classes),1])
    samples.append([get_different_person(classes),1])
    samples.append([get_different_person(classes),1])
  
  random.shuffle(samples)
  return samples

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from pathlib import Path
from torchvision import transforms

In [0]:
indicies = np.arange(len(samples))

np.random.shuffle(indicies)

train_sampler = SubsetRandomSampler(indicies[:int(len(samples)*0.8)])
validation_sampler = SubsetRandomSampler(indicies[int(len(samples)*0.8):])

TrainLoader = DataLoader(samples, batch_size=16, sampler=train_sampler)
ValidationLoader = DataLoader(samples, batch_size=32, sampler=validation_sampler)
dataloaders={'train':TrainLoader, 'val':ValidationLoader}

In [0]:
batch = next(iter(TrainLoader))
print(batch)

[[tensor([226896, 328218, 226098, 409514, 222678, 164343, 215485,  80110,  31677,
         82479, 378242, 228282, 149164, 251812, 414999, 363749]), tensor([226907, 328226, 226126, 257262, 222696, 164357, 386927,  79813, 127912,
         82471, 378237, 380243, 149188, 251819, 405729, 363747])], tensor([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0])]


In [0]:
model_save_path = path+"models/"

In [19]:
from keras.models import Sequential

model = Sequential()

from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=1024))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=2, activation='softmax'))


Using TensorFlow backend.
W0706 07:16:26.462333 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 07:16:26.505311 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 07:16:26.513806 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [20]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

W0706 07:16:28.486331 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
num_of_samples= 1e5

In [0]:
samples=generate_samples(num_of_samples, classes)

In [0]:
x_train = []
y_train = []
i=0
for x in samples:
  a = train_set[x[0][0]]
  
  b = train_set[x[0][1]]
  c = np.append(a,b)
  #if i == 10:
    
    #print(x_train)
  x_train.append(c)
  y = [1, 0]
  if x[1]==1:
    y = [0, 1]
  y_train.append(y)
  i+=1
  

In [0]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [25]:
model.fit(x_train, y_train, epochs=1, batch_size=16)

W0706 07:16:37.813837 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0706 07:16:37.922169 140011292452736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/1
100000/100000 [==============================] - 35s 347us/step - loss: 0.1552 - acc: 0.7721


In [0]:
for i in range(40):
  samples=generate_samples(num_of_samples, classes)
  x_train = []
  y_train = []
  i=0
  for x in samples:
    a = train_set[x[0][0]]

    b = train_set[x[0][1]]
    c = np.append(a,b)
    #if i == 10:

      #print(x_train)
    x_train.append(c)
    y = [1, 0]
    if x[1]==1:
      y = [0, 1]
    y_train.append(y)
    i+=1
  x_train = np.array(x_train)
  y_train = np.array(y_train)
  model.fit(x_train, y_train, epochs=1, batch_size=16)

Epoch 1/1
100000/100000 [==============================] - 18s 178us/step - loss: 0.0580 - acc: 0.9273
Epoch 1/1
100000/100000 [==============================] - 18s 178us/step - loss: 0.0566 - acc: 0.9283
Epoch 1/1
100000/100000 [==============================] - 18s 179us/step - loss: 0.0556 - acc: 0.9301
Epoch 1/1
100000/100000 [==============================] - 18s 178us/step - loss: 0.0569 - acc: 0.9285
Epoch 1/1
100000/100000 [==============================] - 18s 176us/step - loss: 0.0563 - acc: 0.9292
Epoch 1/1
100000/100000 [==============================] - 18s 175us/step - loss: 0.0546 - acc: 0.9315
Epoch 1/1
100000/100000 [==============================] - 17s 174us/step - loss: 0.0543 - acc: 0.9322
Epoch 1/1
100000/100000 [==============================] - 18s 183us/step - loss: 0.0545 - acc: 0.9317
Epoch 1/1
100000/100000 [==============================] - 18s 183us/step - loss: 0.0546 - acc: 0.9318
Epoch 1/1
100000/100000 [==============================] - 18s 183us/step

In [0]:
num_of_test_samples= 3e4

In [0]:
test_samples=generate_samples(num_of_test_samples, classes)

In [0]:
x_test = []
y_test = []
i=0
for x in test_samples:
  a = train_set[x[0][0]]
  
  b = train_set[x[0][1]]
  c = np.append(a,b)
  #if i == 10:
    
    #print(x_train)
  x_test.append(c)
  y = [1, 0]
  if x[1]==1:
    y = [0, 1]
  y_test.append(y)
  i+=1
  

In [0]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [0]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=16)


100000/100000 [==============================] - 7s 69us/step


In [0]:
print(loss_and_metrics)

[0.049621841023666784, 0.93833]


In [0]:
model.save(model_save_path+"model.h5")

In [0]:
distances = []
for i in range(len(x_test)):
  x1=x_test[i][:512]
  x2=x_test[i][512:]
  dis=0
  for j in range(512):
    dis+= (abs(x1[j]-x2[j])/(abs(x1[j])+abs(x2[j])))
  #print(dis)
  #dist = np.sum((x1-x2)**2)
  distances.append(dis)

In [0]:
Max = max(distances)
Min = min(distances)
thresh = (Max+Min)/2
print(Min, Max, thresh)
#print(len(distances))

0.0 417.7330895542989 208.86654477714944


In [0]:
print(distances)

[257.52835199987464, 269.3153749215971, 185.1657457731454, 235.75555700640876, 236.5538485766904, 375.40435551745037, 215.3190637204824, 363.7663604431604, 266.0817567921941, 378.0031613411547, 379.0583464826359, 367.1232306059648, 376.08805798275375, 381.9053436240918, 348.1294324628702, 290.2976456432485, 368.96383455067223, 263.794748498448, 368.2792657619157, 348.0181502590744, 335.6267669324454, 377.41338132478865, 384.1717686949672, 360.2184746875902, 377.7468168497148, 359.748636298464, 358.31122936408946, 266.6150398107601, 356.2065717605409, 333.9462060388535, 294.7476237696069, 332.49767863372114, 333.49331103708596, 277.0295342720429, 241.02299115651053, 371.2135159851801, 383.01488950377717, 234.10627420077154, 245.13410407643389, 356.9124965849485, 376.873678122862, 300.0994665612372, 372.593432943985, 367.03755756391257, 326.77036093709836, 289.65255938737823, 282.50832646352154, 370.03794242902774, 359.5001645801816, 357.39587396958774, 359.88203364555153, 373.9346828429

In [0]:
true_ecl =0
for i in range(len(y_test)):
  if(distances[i]>thresh and y_test[i][1]==1):
    true_ecl+=1
    #print(true_ecl)
  elif(distances[i]<thresh and y_test[i][0]==1):
    true_ecl+=1
    #print(true_ecl)

In [0]:
print(true_ecl)
print(distances[1]<thresh and y_test[1][0]==1)

53058
False


In [0]:
out = model.predict(x_test, batch_size=1)


In [0]:

def get_test_values():
  x_test = []

  for i in range(len(test_pairs)):
    a = test_set[test_pairs["index1"][i]]
    b = test_set[test_pairs["index2"][i]]
    c = np.append(a,b)
    x_test.append(c)
  return x_test

In [0]:
x_test_f = get_test_values()
x_test_f = np.array(x_test_f)

In [0]:
out = model.predict(x_test_f, batch_size=1)


In [0]:
distances = []
for i in range(len(x_test_f)):
  x1=x_test_f[i][:512]
  x2=x_test_f[i][512:]
  dis=0
  for j in range(512):
    dis+= (abs(x1[j]-x2[j])**3/(abs(x1[j])+abs(x2[j]))**4)
  #print(dis)
  #dist = np.sum((x1-x2)**2)
  distances.append(dis)

In [33]:
print((distances))

[110.33858961180546, 327.93259823268903, 144.18610245064917, 298.5972684414061, 166.93620621839975, 141.52834079269857, 143.71648707843192, 135.9851416594473, 152.93510773330385, 169.55944221866503, 165.1974444892885, 186.34365364820346, 139.15528749538626, 242.77498580946113, 130.56122913710968, 146.48014738687024, 155.81430489547245, 149.50637073206352, 170.9144798098654, 194.4001669446459, 178.65104888591642, 139.41059510038747, 430.87971597510534, 156.42666733556163, 145.3704746140076, 140.14099958853745, 291.3455067071833, 162.32250044299352, 169.2469990100526, 163.03187092933476, 167.1307527554877, 167.73112293475512, 175.99403731313578, 184.37487548132054, 218.1892624059461, 144.00900449161713, 149.57246406858303, 130.85709265397617, 130.37367266770545, 132.27576182902996, 143.9245944264595, 165.385675929751, 145.7098989035456, 152.39348869084216, 182.664415079479, 133.18767077761498, 147.02827877381745, 330.5376206403973, 170.06665726854806, 180.8306647657785, 175.5303679730581

In [34]:
Max = max(distances)
Min = min(distances)
thresh = (Max+Min)/2
print(Min, Max, thresh)
#print(len(distances))

89.58480933573809 1012.6556557728484 551.1202325542932


In [0]:
y_final_test = []
for i in range(len(distances)):
  if(distances[i]>thresh):
    y_final_test.append([0, 1])
    #print(true_ecl)
  else:
    y_final_test.append([1, 0])
    #print(true_ecl)

In [40]:
diff = [x for x in range(len(distances)) if y_final_test[x][1]==1]

print(diff)

[3638, 5453, 6946, 8595, 11021, 12290, 13473, 15931, 17497, 17689, 17801, 18358, 19471, 20231, 22348, 22357, 23075, 23620, 27599, 27928, 29746, 30615, 30770]


In [0]:
import math
import random
out_scale = []
for i in range(len(out)):
  out_scale.append(np.argmax(out[i]))


In [0]:
print(distances)

[110.33858961180546, 327.93259823268903, 144.18610245064917, 298.5972684414061, 166.93620621839975, 141.52834079269857, 143.71648707843192, 135.9851416594473, 152.93510773330385, 169.55944221866503, 165.1974444892885, 186.34365364820346, 139.15528749538626, 242.77498580946113, 130.56122913710968, 146.48014738687024, 155.81430489547245, 149.50637073206352, 170.9144798098654, 194.4001669446459, 178.65104888591642, 139.41059510038747, 430.87971597510534, 156.42666733556163, 145.3704746140076, 140.14099958853745, 291.3455067071833, 162.32250044299352, 169.2469990100526, 163.03187092933476, 167.1307527554877, 167.73112293475512, 175.99403731313578, 184.37487548132054, 218.1892624059461, 144.00900449161713, 149.57246406858303, 130.85709265397617, 130.37367266770545, 132.27576182902996, 143.9245944264595, 165.385675929751, 145.7098989035456, 152.39348869084216, 182.664415079479, 133.18767077761498, 147.02827877381745, 330.5376206403973, 170.06665726854806, 180.8306647657785, 175.5303679730581

In [0]:
baseline = pd.DataFrame({'Predicted':distances})
baseline['Id'] = range(len(baseline))

baseline.to_csv(path+'out.csv',index=False)

In [0]:
def apply_test():
  num_of_test_samples= 1e5
  test_samples=generate_samples(num_of_test_samples, classes)
  x_test = []
  y_test = []
  i=0
  for x in test_samples:
    a = train_set[x[0][0]]

    b = train_set[x[0][1]]
    c = np.append(a,b)
    #if i == 10:

      #print(x_train)
    x_test.append(c)
    y = 0
    if x[1]==1:
      y = 1
    y_test.append(y)
    i+=1
    
  x_test = np.array(x_test)
  y_test = np.array(y_test)
  
  out = model.predict(x_test, batch_size=1)
  out_scale = []
  for i in range(len(out)):
    out_scale.append(np.argmax(out[i]))
  
  return y_test, out_scale


In [0]:
def get_params(y, out):
  TN=0
  TP=0
  FN=0
  FP=0
  for i in range(len(y)):
    #print(y[i], out[i])
    if(y[i]==0 and out[i]==0):
      TP+=1
    elif(y[i]==0 and out[i]==1):
      FN+=1
    elif(y[i]==1 and out[i]==1):
      TN+=1
    else:
      FP+=1
  return TP, FN, TN, FP

In [0]:
y,out = apply_test()

In [0]:
TP, FN, TN, FP = get_params(y, out)

In [0]:
print(TP, FN, TN, FP)

45494 4506 48176 1824


In [0]:
print(1-(TN/(TN+FP)))
print(TP/(TP+FN))

0.03647999999999996
0.90988
